# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [657]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from razdel import tokenize

from IPython.display import Image
from IPython.core.display import HTML

In [658]:
data = pd.read_csv('labeled.csv')
data.toxic.value_counts(normalize=True)

toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64

In [659]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [660]:
def razdel_tokenize(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens]

In [661]:
vectorizer_def = TfidfVectorizer(min_df=10, max_df=0.3)
vectorizer_raz = TfidfVectorizer(min_df=10, max_df=0.3, tokenizer=razdel_tokenize)

In [662]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [663]:
default_X = vectorizer_def.fit_transform(train.comment)
default_X_test = vectorizer_def.transform(test.comment)

razdel_X = vectorizer_raz.fit_transform(train.comment)
razdel_X_test = vectorizer_raz.transform(test.comment)

C:\Users\robst\PycharmProjects\HSE_NLP\venv\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [664]:
default_X.shape, default_X_test.shape

((12970, 3380), (1442, 3380))

In [665]:
razdel_X.shape, razdel_X_test.shape

((12970, 3437), (1442, 3437))

In [666]:
y = train.toxic.values
y_test = test.toxic.values

In [667]:
clf_def = LogisticRegression(C=0.1, class_weight='balanced')
clf_raz = LogisticRegression(C=0.1, class_weight='balanced')

In [668]:
clf_def.fit(default_X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [669]:
clf_raz.fit(razdel_X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [670]:
preds_def = clf_def.predict(default_X_test)

In [671]:
preds_raz = clf_raz.predict(razdel_X_test)

In [672]:
preds_def

array([1., 0., 1., ..., 0., 0., 0.])

In [673]:
preds_raz

array([1., 0., 1., ..., 0., 0., 0.])

In [674]:
print(classification_report(y_test, preds_def, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.84      0.86       985
         1.0       0.69      0.77      0.73       457

    accuracy                           0.82      1442
   macro avg       0.79      0.80      0.79      1442
weighted avg       0.82      0.82      0.82      1442



In [675]:
print(classification_report(y_test, preds_raz, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.84      0.86       985
         1.0       0.69      0.76      0.72       457

    accuracy                           0.82      1442
   macro avg       0.79      0.80      0.79      1442
weighted avg       0.82      0.82      0.82      1442



In [676]:
probas_def = clf_def.predict_proba(default_X_test)
probas_raz = clf_raz.predict_proba(razdel_X_test)

In [677]:
probas_def

array([[0.49453097, 0.50546903],
       [0.5828941 , 0.4171059 ],
       [0.27883016, 0.72116984],
       ...,
       [0.70526961, 0.29473039],
       [0.61567224, 0.38432776],
       [0.6194527 , 0.3805473 ]])

In [678]:
probas_raz

array([[0.49143227, 0.50856773],
       [0.58355404, 0.41644596],
       [0.30183449, 0.69816551],
       ...,
       [0.59878933, 0.40121067],
       [0.6234262 , 0.3765738 ],
       [0.5797565 , 0.4202435 ]])

In [679]:
preds_def = (probas_def[:,1]>0.85).astype(int)
preds_raz = (probas_raz[:,1]>0.85).astype(int)

In [680]:
print(classification_report(y_test, preds_def, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       985
         1.0       0.95      0.05      0.09       457

    accuracy                           0.70      1442
   macro avg       0.82      0.52      0.45      1442
weighted avg       0.78      0.70      0.59      1442



In [681]:
print(classification_report(y_test, preds_raz, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       985
         1.0       0.92      0.05      0.10       457

    accuracy                           0.70      1442
   macro avg       0.81      0.53      0.46      1442
weighted avg       0.77      0.70      0.59      1442



## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картинке с пояснениями на английском.
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления.

In [682]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from scipy.sparse.csr import csr_matrix

C:\Users\robst\AppData\Local\Temp\ipykernel_9584\3164112859.py:4: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix


In [683]:
def razdel_tokenize_fix(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens if _.text != ","]

In [684]:
vec = TfidfVectorizer(tokenizer=razdel_tokenize_fix)

In [685]:
texts = [
    "я и ты",
    "ты и я",
    "я, я и только я",
    "только не я",
    "он"
]

In [686]:
table = vec.fit_transform(texts)
print(table)

  (0, 4)	0.6778032959469461
  (0, 0)	0.5626379641131365
  (0, 5)	0.47330879280030447
  (1, 4)	0.6778032959469461
  (1, 0)	0.5626379641131365
  (1, 5)	0.47330879280030447
  (2, 3)	0.4056325249335055
  (2, 0)	0.33671163798017656
  (2, 5)	0.8497573345166112
  (3, 1)	0.7127752157729959
  (3, 3)	0.5750625560879445
  (3, 5)	0.4015651234424611
  (4, 2)	1.0


C:\Users\robst\PycharmProjects\HSE_NLP\venv\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0 = sentence no.
4 = word index
0.6778032959469461 = tf-idf weight

In [687]:
for i, feature in enumerate(vec.get_feature_names_out()):
    print(i, feature)
feature_names = vec.get_feature_names_out()

0 и
1 не
2 он
3 только
4 ты
5 я


In [688]:
df = pd.DataFrame(table.toarray(), columns = vec.get_feature_names_out(), index=texts)
print(df)

                        и        не   он    только        ты         я
я и ты           0.562638  0.000000  0.0  0.000000  0.677803  0.473309
ты и я           0.562638  0.000000  0.0  0.000000  0.677803  0.473309
я, я и только я  0.336712  0.000000  0.0  0.405633  0.000000  0.849757
только не я      0.000000  0.712775  0.0  0.575063  0.000000  0.401565
он               0.000000  0.000000  1.0  0.000000  0.000000  0.000000


    ## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г) f1 мера каждого из классификаторов должна быть минимум 0.75

*random_seed не считается за параметр

In [689]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from razdel import tokenize

In [690]:
def razdel_tokenize(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens]

In [691]:
data = pd.read_csv('labeled.csv')
data.toxic.value_counts(normalize=True)

toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64

In [692]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [693]:
vec_count = CountVectorizer(min_df=8, max_df=0.3, tokenizer=razdel_tokenize, lowercase=True, ngram_range=(1,4), strip_accents='unicode')
vec_tfidf = TfidfVectorizer(min_df=8, max_df=0.3, tokenizer=razdel_tokenize, lowercase=True, ngram_range=(1,5), strip_accents='unicode')

In [694]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [695]:
count_X = vec_count.fit_transform(train.comment)
count_X_test = vec_count.transform(test.comment)

tfidf_X = vec_tfidf.fit_transform(train.comment)
tfidf_X_test = vec_tfidf.transform(test.comment)

C:\Users\robst\PycharmProjects\HSE_NLP\venv\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [696]:
count_X.shape, count_X_test.shape

((12970, 9207), (1442, 9207))

In [697]:
tfidf_X.shape, tfidf_X_test .shape

((12970, 9221), (1442, 9221))

In [698]:
y = train.toxic.values
y_test = test.toxic.values

In [699]:
dec_class = MultinomialNB(alpha=0.1)
dec_class.fit(count_X, y)
dec_preds = dec_class.predict(count_X_test)

print(classification_report(y_test, dec_preds))


              precision    recall  f1-score   support

         0.0       0.86      0.87      0.86       953
         1.0       0.74      0.72      0.73       489

    accuracy                           0.82      1442
   macro avg       0.80      0.80      0.80      1442
weighted avg       0.82      0.82      0.82      1442



In [700]:
kneighbors_class = LogisticRegression(C=0.6, class_weight='balanced')
kneighbors_class.fit(tfidf_X, y)
kneighbors_preds = kneighbors_class.predict(tfidf_X_test)

print(classification_report(y_test, kneighbors_preds))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       953
         1.0       0.73      0.78      0.75       489

    accuracy                           0.83      1442
   macro avg       0.81      0.82      0.81      1442
weighted avg       0.83      0.83      0.83      1442



## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [701]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
import re
russian_stopwords = stopwords.words('russian')

In [702]:
def razdel_tokenize(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens if _.text not in russian_stopwords]

In [703]:
data = pd.read_csv('labeled.csv')
data.toxic.value_counts(normalize=True)

toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64

In [704]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.3, tokenizer=razdel_tokenize, stop_words=list(russian_stopwords))

In [705]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [706]:
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

C:\Users\robst\PycharmProjects\HSE_NLP\venv\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [707]:
X.shape, X_test.shape

((12970, 3304), (1442, 3304))

In [708]:
y = train.toxic.values
y_test = test.toxic.values

In [709]:
lr = LogisticRegression(C=0.1, class_weight='balanced')
lr.fit(X, y)
lr_preds = lr.predict(X_test)

rf = RandomForestClassifier(n_estimators=100, max_depth=20, )
rf.fit(X, y)
rf_preds = rf.predict(X_test)

In [710]:
# важность признаков
lr_feature_importance = lr.coef_[0]
rf_feature_importance = rf.feature_importances_

In [711]:
vec_feature_names = vectorizer.get_feature_names_out()

In [712]:
features = list(zip(vec_feature_names, lr_feature_importance))
features = [f for f in features if re.match(r'^\w+$', f[0])]
features.sort(key=lambda f: f[1], reverse=True)
top_features = features[:5]
print("Топ-5 токсичных слов для LogisticRegression:", top_features)

Топ-5 токсичных слов для LogisticRegression: [('тебе', 1.727446496695935), ('хохлы', 1.6180118650552706), ('хохлов', 1.5549590892173057), ('нахуй', 1.3497672331746935), ('пиздец', 1.1376157209129165)]


In [713]:
features = list(zip(vec_feature_names, rf_feature_importance))
features = [f for f in features if re.match(r'^\w+$', f[0])]
features.sort(key=lambda f: f[1], reverse=True)
top_features = features[:5]
print("Топ-5 токсичных слов для RandomForest:", top_features)

Топ-5 токсичных слов для RandomForest: [('тебе', 0.03190231577976939), ('хохлы', 0.02754997733951765), ('хохлов', 0.023991852220358902), ('нахуй', 0.019344975123603628), ('блять', 0.017599437908520723)]


Без фильтрации

In [714]:
# топ-5
top_five_lr_features = [vec_feature_names[i] for i in lr_feature_importance.argsort()[::-1][:5]]
top_five_rf_features = [vec_feature_names[i] for i in rf_feature_importance.argsort()[::-1][:5]]

In [715]:
print("Топ-5 токсичных слов для LogisticRegression:", top_five_lr_features)

Топ-5 токсичных слов для LogisticRegression: ['тебе', 'хохлы', 'хохлов', '!', 'нахуй']


In [716]:
print("Топ-5 токсичных слов для RandomForest:", top_five_rf_features)

Топ-5 токсичных слов для RandomForest: [')', '!', 'тебе', 'хохлы', 'хохлов']
